In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F 

In [3]:
spark = SparkSession.builder.getOrCreate()

#Create DataFrame with match details
data = [
    ("India", "Australia", "India"),
    ("England", "Pakistan", "England"),
    ("South Africa", "New Zealand", "New Zealand"),
    ("West Indies", "Sri Lanka", "Sri Lanka"),
    ("India", "Pakistan", "India"),
    ("Australia", "England", "Australia"),
    ("New Zealand", "Sri Lanka", "New Zealand"),
    ("South Africa", "West Indies", "South Africa"),
    ("India", "England", "India"),
    ("Pakistan", "Australia", "Pakistan"),
]

columns = ["Team_1", "Team_2", "Winner"]

df = spark.createDataFrame(data, columns)


In [4]:
df.show()

+------------+-----------+------------+
|      Team_1|     Team_2|      Winner|
+------------+-----------+------------+
|       India|  Australia|       India|
|     England|   Pakistan|     England|
|South Africa|New Zealand| New Zealand|
| West Indies|  Sri Lanka|   Sri Lanka|
|       India|   Pakistan|       India|
|   Australia|    England|   Australia|
| New Zealand|  Sri Lanka| New Zealand|
|South Africa|West Indies|South Africa|
|       India|    England|       India|
|    Pakistan|  Australia|    Pakistan|
+------------+-----------+------------+



In [30]:
df_unioned = df.select(col('Team_1').alias('team'),'Winner').union(df.select(col('Team_2').alias('team'),'Winner'))\
                .withColumn('winner_flag',when(col('team') == col('Winner'),1).otherwise(0))

In [32]:
df_unioned.show(5)

df_unioned.groupBy('team').agg(count('team').alias('total_matches'),sum('winner_flag')).show()

+------------+-----------+-----------+
|        team|     Winner|winner_flag|
+------------+-----------+-----------+
|       India|      India|          1|
|     England|    England|          1|
|South Africa|New Zealand|          0|
| West Indies|  Sri Lanka|          0|
|       India|      India|          1|
+------------+-----------+-----------+
only showing top 5 rows



+------------+-------------+----------------+
|        team|total_matches|sum(winner_flag)|
+------------+-------------+----------------+
|   Sri Lanka|            2|               1|
|       India|            3|               3|
| West Indies|            2|               0|
| New Zealand|            2|               2|
|     England|            3|               1|
|   Australia|            3|               1|
|South Africa|            2|               1|
|    Pakistan|            3|               1|
+------------+-------------+----------------+



In [34]:
df_unioned.groupBy('team').agg(count('team').alias('total_matches'),sum('winner_flag')).explain()

== Physical Plan ==
*(4) HashAggregate(keys=[team#198], functions=[count(team#198), sum(cast(winner_flag#208 as bigint))])
+- Exchange hashpartitioning(team#198, 200), true, [id=#590]
   +- *(3) HashAggregate(keys=[team#198], functions=[partial_count(team#198), partial_sum(cast(winner_flag#208 as bigint))])
      +- Union
         :- *(1) Project [Team_1#0 AS team#198, CASE WHEN (Team_1#0 = Winner#2) THEN 1 ELSE 0 END AS winner_flag#208]
         :  +- *(1) Scan ExistingRDD[Team_1#0,Team_2#1,Winner#2]
         +- *(2) Project [Team_2#1 AS team#204, CASE WHEN (Team_2#1 = Winner#2) THEN 1 ELSE 0 END AS winner_flag#308]
            +- *(2) Scan ExistingRDD[Team_1#0,Team_2#1,Winner#2]


